In [1]:
import pandas as pd
import json

In [2]:
with open('./model_abtest.log','r') as f:
    raw = f.read().split('\n')[:-1]

In [3]:
data_list = []
for event in raw:
    obs = json.loads(event)
    obs['extra_data'] = obs.get('extra_data',None)
    data_list.append(obs)

In [4]:
df = pd.DataFrame.from_dict(data_list) 
df

,name,time,salt,inputs,params,event,checksum,extra_data
0,ModelExperiment,1645927283,ModelExperiment,{'userid': 521},"{'use_pytorch': 1, 'model_type': 'BERT'}",exposure,53e479ee,None
1,ModelExperiment,1645927452,ModelExperiment,{'userid': 521},"{'use_pytorch': 1, 'model_type': 'BERT'}",rate,53e479ee,{'rate_val': 5}
2,ModelExperiment,1645927456,ModelExperiment,{'userid': 521},"{'use_pytorch': 1, 'model_type': 'BERT'}",exposure,53e479ee,None
3,ModelExperiment,1645927580,ModelExperiment,{'userid': 927},"{'use_pytorch': 0, 'model_type': 'GPT-3'}",exposure,53e479ee,None
4,ModelExperiment,1645927660,ModelExperiment,{'userid': 927},"{'use_pytorch': 0, 'model_type': 'GPT-3'}",rate,53e479ee,{'rate_val': 9}
5,ModelExperiment,1645927663,ModelExperiment,{'userid': 927},"{'use_pytorch': 0, 'model_type': 'GPT-3'}",exposure,53e479ee,None
6,ModelExperiment,1645927717,ModelExperiment,{'userid': 369},"{'use_pytorch': 1, 'model_type': 'BERT'}",exposure,53e479ee,None
7,ModelExperiment,1645927772,ModelExperiment,{'userid': 369},"{'use_pytorch': 1, 'model_type': 'BERT'}",rate,53e479ee,{'rate_val': 6}
8,ModelExperiment,1645927773,ModelExperiment,{'userid': 369},"{'use_pytorch': 1, 'model_type': 'BERT'}",exposure,53e479ee,None
9,ModelExperiment,1645929681,ModelExperiment,{'userid': 369},"{'use_pytorch': 1, 'model_type': 'BERT'}",exposure,53e479ee,None


In [5]:
exp_df = df.loc[df.event=='rate',['params','extra_data']]
exp_df['variant'] = exp_df['params'].apply(lambda x: x['use_pytorch'])
exp_df['rating'] = exp_df['extra_data'].apply(lambda x: x['rate_val'])
exp_df

,params,extra_data,variant,rating
1,"{'use_pytorch': 1, 'model_type': 'BERT'}",{'rate_val': 5},1,5
4,"{'use_pytorch': 0, 'model_type': 'GPT-3'}",{'rate_val': 9},0,9
7,"{'use_pytorch': 1, 'model_type': 'BERT'}",{'rate_val': 6},1,6
11,"{'use_pytorch': 0, 'model_type': 'GPT-3'}",{'rate_val': 10},0,10


In [6]:
d1 = exp_df.loc[exp_df.variant==0,'rating'].values
d2 = exp_df.loc[exp_df.variant==1,'rating'].values

In [7]:
d1

array([ 9, 10])

In [8]:
d2

array([5, 6])

In [9]:
import scipy.stats
statistic, pval = scipy.stats.ttest_ind(d1,d2, alternative="greater")
print(statistic,pval)

5.65685424949238 0.014928749927334055
